In [13]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

In [14]:
# === CONFIGURATION ===
VIDEO_PATH          = "gaze.mp4"
OUTPUT_VIDEO_PATH   = "annotated_gaze.mp4"
ANNOTATE_VIDEO      = True    # Set False to skip writing the annotated video
GAZE_STATIONARY_TH  = 0.02    # Max Δ in normalized gaze to count as “stationary” in 1s
NUM_BINS            = 20      # Heatmap resolution on horizontal axis

In [15]:
# === INIT MEDIAPIPE FACE MESH ===
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    refine_landmarks=True,            # includes iris landmarks
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils
mp_styles  = mp.solutions.drawing_styles

In [16]:
# === OPEN VIDEO ===
cap       = cv2.VideoCapture(0)
fps       = cap.get(cv2.CAP_PROP_FPS)
W, H      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_cnt = 0

In [17]:
# === SET UP WRITER IF ANNOTATING ===
if ANNOTATE_VIDEO:
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (W, H))

In [18]:
# === STORAGE ===
gaze_positions   = []   # list of tuples: (second, norm_x, norm_y)
gaze_summary     = []   # per-second summary
horizontal_bins  = np.zeros(NUM_BINS)

In [19]:
# === PROCESS FRAMES ===
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_cnt += 1
    sec = int(frame_cnt / fps)

    # 1) FACE MESH DETECTION
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = face_mesh.process(rgb)

    if res.multi_face_landmarks:
        lm = res.multi_face_landmarks[0]

        # 2) EXTRACT IRIS LANDMARKS (left & right eyes)
        # left iris indices: [468–472], right iris: [473–477]
        left_iris_pts  = np.array([(lm.landmark[i].x, lm.landmark[i].y) for i in range(468, 473)])
        right_iris_pts = np.array([(lm.landmark[i].x, lm.landmark[i].y) for i in range(473, 478)])
        iris_center    = np.vstack((left_iris_pts, right_iris_pts)).mean(axis=0)

        norm_x, norm_y = iris_center  # normalized [0,1] coordinates
        gaze_positions.append((sec, norm_x, norm_y))

        # 3) OPTIONAL ANNOTATION
        if ANNOTATE_VIDEO:
            # draw full face mesh
            mp_drawing.draw_landmarks(
                frame, lm, mp_face.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_styles.get_default_face_mesh_tesselation_style()
            )
            # draw a green dot at gaze point
            px, py = int(norm_x * W), int(norm_y * H)
            cv2.circle(frame, (px, py), 5, (0, 255, 0), -1)
    else:
        gaze_positions.append((sec, None, None))

    # 4) WRITE ANNOTATED FRAME
    if ANNOTATE_VIDEO:
        writer.write(frame)

KeyboardInterrupt: 

In [9]:
# cleanup
cap.release()
if ANNOTATE_VIDEO:
    writer.release()
    print(f"✅ Annotated video saved at {OUTPUT_VIDEO_PATH}")

✅ Annotated video saved at annotated_gaze.mp4


In [10]:
# === AGGREGATE PER-SECOND GAZE AND STATIONARY FLAG ===
total_secs = int(frame_cnt / fps)
for s in range(total_secs):
    xs = [x for (t, x, y) in gaze_positions if t == s and x is not None]
    ys = [y for (t, x, y) in gaze_positions if t == s and y is not None]

    if xs:
        avg_x, avg_y = np.mean(xs), np.mean(ys)
        # determine if gaze stayed within small window
        stationary = (max(xs)-min(xs) < GAZE_STATIONARY_TH) and (max(ys)-min(ys) < GAZE_STATIONARY_TH)
    else:
        avg_x = avg_y = None
        stationary = False

    gaze_summary.append({
        "second": s,
        "avg_x": avg_x,
        "avg_y": avg_y,
        "stationary": stationary
    })

    # build horizontal heatmap
    if avg_x is not None:
        bin_idx = int(avg_x * (NUM_BINS - 1))
        horizontal_bins[bin_idx] += 1


In [11]:
# === PRINT SUMMARY ===
print("\nSecond |   Avg X   |   Avg Y   | Stationary")
print("------------------------------------------")
for m in gaze_summary:
    ax = f"{m['avg_x']:.2f}" if m['avg_x'] is not None else "  –  "
    ay = f"{m['avg_y']:.2f}" if m['avg_y'] is not None else "  –  "
    flag = "Yes" if m["stationary"] else "No"
    print(f"{m['second']:>6} | {ax:^9} | {ay:^9} | {flag:^10}")


Second |   Avg X   |   Avg Y   | Stationary
------------------------------------------
     0 |   0.43    |   0.19    |    Yes    
     1 |   0.42    |   0.18    |    Yes    
     2 |   0.41    |   0.20    |     No    
     3 |   0.42    |   0.22    |    Yes    
     4 |   0.42    |   0.20    |     No    
     5 |   0.43    |   0.21    |    Yes    
     6 |   0.40    |   0.20    |     No    
     7 |   0.46    |   0.19    |     No    
     8 |   0.44    |   0.19    |     No    
     9 |   0.38    |   0.19    |    Yes    
    10 |   0.39    |   0.20    |     No    
    11 |   0.42    |   0.18    |     No    
    12 |   0.42    |   0.16    |     No    
    13 |   0.41    |   0.22    |     No    
    14 |   0.41    |   0.20    |     No    
    15 |   0.40    |   0.20    |     No    
    16 |   0.40    |   0.21    |    Yes    
    17 |   0.41    |   0.22    |    Yes    
    18 |   0.41    |   0.19    |     No    
    19 |   0.41    |   0.20    |     No    
    20 |   0.40    |   0.22    |